### Cài đặt thư viện unsloth

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Tải model từ unsloth để phục vụ cho việc fine tuning

In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import  AutoTokenizer

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Import file csv và format dataset
Import file csv và format dataset về định dạng dùng để fine tuning

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]
    inputs       = examples["Input"]
    outputs      = examples["Output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
from google.colab import files
file_uploaded=files.upload()
import io
import pandas as pd


uploaded_filename = next(iter(file_uploaded))

# Read the CSV file into a DataFrame
df = pd.read_csv(io.BytesIO(file_uploaded[uploaded_filename]))

# Kiểm tra 3 dòng đầu tiên của DataFrame
print(df.head(3))

# Bước 3: Chuyển DataFrame thành Hugging Face Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True,)
for i in range(3):
    print(dataset[i])
    print("-" * 20)

Saving FineTuningMetaDAP - Sheet1 (1).csv to FineTuningMetaDAP - Sheet1 (1).csv
                             Instruction                               Input  \
0             Giải thích ý nghĩa câu hỏi               Giới thiệu về MetaDAP   
1  Liệt kê văn bản liên quan đến nhân sự        Nhân sự công ty gồm những ai   
2                     Liệt kê ban cố vấn  MetaDAP có các ban cố gồm những ai   

                                              Output  
0  MetaDAP là một công ty khởi nghiệp công nghệ, ...  
1  TS. Trần Quý là Viện trưởng Viện Phát Triển Ki...  
2  TS. Phạm Hồng Hải cố vấn phát triển bền vững g...  


Map:   0%|          | 0/261 [00:00<?, ? examples/s]

{'Instruction': 'Giải thích ý nghĩa câu hỏi', 'Input': 'Giới thiệu về MetaDAP', 'Output': 'MetaDAP là một công ty khởi nghiệp công nghệ, phát triển nền tảng tài sản số dành cho doanh nghiệp (Enterprise Blockchain). Nền tảng này giúp doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của blockchain nhưng vẫn đảm bảo tuân thủ các quy định pháp lý. Người dùng có thể trực tiếp quản lý và giao dịch các tài sản có giá trị, bao gồm tài sản hữu hình và vô hình, ở cả thị trường sơ cấp và thứ cấp. Toàn bộ quy trình được thực hiện tự động và trực tuyến mà không cần sự tham gia của các bên trung gian, giúp tăng tính khả dụng và thanh khoản của tài sản.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiải thích ý nghĩa câu hỏi\n\n### Input:\nGiới thiệu về MetaDAP\n\n### Response:\nMetaDAP là một công ty khởi nghiệp công nghệ, phát triển nền tảng tài

## Test model trước khi train

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Giới thiệu công ty MetaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


MetaDAP (Meta Data Analysis Platform) is a company that specializes in providing data analysis and visualization solutions to businesses and organizations. Here's an overview of the company:

**Mission**: MetaDAP's mission is to empower organizations to make data-driven decisions by providing them with easy-to-use data analysis and visualization tools.

**Services**: MetaDAP offers a range of services, including:

1. **Data Analysis**: MetaDAP's team of experts helps clients analyze their data to identify trends, patterns, and correlations.
2. **Data Visualization**: MetaDAP creates interactive and dynamic visualizations to help clients understand and communicate their data insights.
3. **Business Intelligence**: MetaDAP helps clients develop business intelligence solutions to improve decision-making and drive business growth.
4. **Data Science**: MetaDAP's data scientists work with clients to develop predictive models, machine learning algorithms, and other advanced analytics solution

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Giới thiệu nhân sự MetaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 500, pad_token_id = tokenizer.eos_token_id)

MetaDAP is a team of experts in the field of Digital Asset Protection (DAP). The team is dedicated to providing cutting-edge solutions to protect digital assets, including cryptocurrencies, NFTs, and other digital valuables. Here are some key members of the MetaDAP team:

**Dr. Leonardo "Leo" D'Angelo**: CEO and Founder of MetaDAP. Dr. D'Angelo is a renowned expert in cryptography and digital security. He has over 20 years of experience in the field and has published numerous papers on cryptography and digital asset protection.

**Dr. Sophia Patel**: CTO of MetaDAP. Dr. Patel is a leading expert in blockchain technology and digital asset security. She has a strong background in computer science and has worked on several high-profile blockchain projects.

**Dr. James "Jim" Thompson**: Head of Research and Development. Dr. Thompson is a respected expert in artificial intelligence and machine learning. He has a strong background in computer science and has worked on several AI-powered pro

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Tài sản số là gì"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

Tài sản số (Digital Asset) là một khái niệm rộng được sử dụng để miêu tả các loại tài sản hoặc giá trị có thể được lưu trữ, trao đổi, hoặc sử dụng trên các nền tảng số (online) và được xác định bằng một dạng mã số (digital identifier) hoặc một địa chỉ IP (IP address).

Tài sản số có thể bao gồm:

1. Tiền điện tử (Cryptocurrency): các loại tiền tệ kỹ thuật số như Bitcoin, Ethereum, Litecoin, etc.
2. Token: các loại token được phát hành trên các nền tảng blockchain, như token của các dự án ICO (Initial Coin Offering).
3. NFT (Non-Fungible Token): các loại token không thể được thay thế, được sử dụng để đại diện cho các tài sản hữu hình, như nghệ thuật, nhạc, hoặc video.
4. Dữ liệu số: các loại dữ liệu được lưu trữ trên các nền tảng số, như dữ liệu cá nhân, dữ liệu kinh doanh, hoặc dữ liệu khoa học.
5. Dữ liệu blockchain: các loại dữ liệu được lưu trữ trên các nền tảng blockchain, như sổ cái blockchain, hoặc các loại dữ liệu được sử dụng để tạo ra các token.
6. Ứng dụng di động: các loại ứ

<a name="Train"></a>
# Huấn luyện mô hình
Huấn luyện mô hình sử dụng phương pháp SFT (Supervised fine tuning). Bạn có thể tham khảo thêm tại (https://huggingface.co/docs/trl/sft_trainer).

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = data_collator,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/261 [00:00<?, ? examples/s]

In [ ]:
# @title Hiển thị thông tin hệ thống trước khi train
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.088 GB of memory reserved.


## Tiến hành train

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 261 | Num Epochs = 5 | Total steps = 160
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.836100
2,3.047700
3,2.973000
4,2.697700
5,2.582600
6,2.469500
7,2.227100
8,2.374800
9,2.080600
10,1.908900


## Hiển thị thông tin hệ thống sau khi train

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} giây dùng cho việc huấn luyện mô hình")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} phút dùng cho việc huấn luyện mô hình"
)
print(f"Dung lượng bộ nhớ đỉnh sử dụng = {used_memory} GB.")
print(f"Dung lượng bộ nhớ đỉnh trong khi train = {used_memory_for_lora} GB.")
print(f"Dung lượng bố nhớ đỉnh so với hệ thống = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1639.0335 giây dùng cho việc huấn luyện mô hình
27.32 phút dùng cho việc huấn luyện mô hình
Dung lượng bộ nhớ đỉnh sử dụng = 8.029 GB.
Dung lượng bộ nhớ đỉnh trong khi train = 1.941 GB.
Dung lượng bố nhớ đỉnh so với hệ thống = 54.467 %.
Peak reserved memory for training % of max memory = 13.167 %.


<a name="Thử nghiệm sau fine tuning"></a>
## Thử nghiệm sau khi train
Thử nghiệm model sau khi fine tuning

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Giới thiệu công ty MetaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

MetaDAP (Meta Digital Asset Platform) là một công ty khởi nghiệp công nghệ (Tech-driven startup), MetaDAP phát triển nền tảng tài sản số dành cho doanh nghiệp (Enterprise Blockchain), cho phép doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của blockchain nhưng vẫn đảm bảo tuân thủ pháp lý, từ đó người dùng có thể trực tiếp quản lý và giao dịch sơ cấp (Primary market) lẫn thứ cấp (Secondary market) các tài sản có giá trị (vô hình và hữu hình) tự động, trực tuyến mà không cần sự tham gia của các bên trung gian.

Với nền tảng này, doanh nghiệp có thể trực tiếp quản lý và giao dịch các tài sản như cổ phần, quyền sở hữu, quyền tác giả/sáng chế, giấy phép, hợp đồng và nhiều hơn, trực tuyến mà không cần sự tham gia của các bên trung gian, giúp tiết kiệm hơn 90% thời gian, 50% chi phí, tăng tính khả dụng, thanh khoản của tài sản và giúp tạo ra các mô hình kinh doanh mới đa dạng và linh hoạt.

MetaDAP được phát triển từ năm 2021 bởi một nhóm các chuyên gia công nghệ, pháp lý và tà

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Tại sao chọn metaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

MetaDAP là một công ty khởi nghiệp công nghệ, phát triển nền tảng tài sản số dành cho doanh nghiệp (Enterprise Blockchain). Nền tảng này giúp doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của blockchain nhưng vẫn đảm bảo tuân thủ các quy định pháp lý. Dưới đây là những lý do chính pourquoi các doanh nghiệp lựa chọn MetaDAP:

### Tiết kiệm thời gian: 

Không cần phải xây dựng nền tảng blockchain từ đầu, MetaDAP đã có nền tảng hoàn chỉnh với các tiện ích đã được tích hợp,.doanh nghiệp chỉ cần tập trung vào việc xây dựng ứng dụng của mình.

### Tiếp cận cộng đồng toàn thế giới: 

Nền tảng MetaDAP mở ra cánh cửa tiếp cận cộng đồng toàn thế giới, không chỉ trong nước mà còn trên phạm vi toàn cầu, tạo điều kiện thuận lợi để mở rộng cơ sở khách hàng và tăng sự hình thành các đối tác mới.

### Tiết kiệm chi phí: 

Không cần đầu tư hàng trăm nghìn dollar để mua sắm các dòng GPU cao cấp như AMD Radeon RX 6800 XT hay NVIDIA GeForce RTX 3080 để đào blockchain, MetaDAP cung cấp nền t

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Blockchain là gì"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

Blockchain là công nghệ lưu trữ phân tán, được phát triển nhằm mục đích đảm bảo sự minh bạch, bất biến và khả năng truy vết lịch sử cập nhật của dữ liệu. Do đó, blockchain có các tính chất như một cuốn sổ ghi chép vật lý (mọi cập nhật thay đổi đều để lại dấu vết) nhưng tồn tại dưới dạng kỹ thuật số. Do đó, blockchain còn được gọi là công nghệ sổ cái phân tán (Distributed Ledger Technology - DTL), cung cấp giá trị cố lỗi nhất là minh bạch, chính xác của dữ liệu, làm tiền đề để triển khai hợp đồng thông tinh (Smart Contract) và tinh gọn các qui trình khi không cần phải yêu cầu xác thực dữ liệu từ các bên thứ 3.<|eot_id|>


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Tài sản số là gì"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

Tài sản số là những tài sản được tạo ra bằng Smart Contract, trên công nghệ Blockchain. Tài sản số đại diện cho quyền sở hữu đối với tài sản hữu hình hoặc vô hình tương ứng trong thế giới thực. Tài sản số được tạo ra theo cách đặc biệt để phản ánh tính chất của tài sản, có nghĩa là chúng tồn tại duy nhất và chỉ thuộc về một chủ thể duy nhất tại một thời điểm cụ thể.

Tài sản số có các tính chất như:

1. Độc quyền: Một tài sản số chỉ thuộc về một chủ thể duy nhất.
2. Tồn tại duy nhất: Tài sản số không có dạng vật lý, tồn tại duy nhất trên blockchain.
3. Phân phối trực tuyến: Tài sản số được tạo ra và phân phối trực tuyến thông qua smart contract.
4. Quy trình cập nhật thay đổi: Tài sản số chỉ được cập nhật, thay đổi duy nhất một lần, tại thời điểm tạo ra.

Tài sản số được sử dụng để triển khai các hợp đồng thông tinh (Smart Contract) có thể tự động thực thi các điều khoản nếu các bên không tuân thủ, giúp tối ưu và tinh gọn các quy trình hoạt động của doanh nghiệp.<|eot_id|>


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Tầm nhìn và sứ mệnh của MetaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

MetaDAP có tầm nhìn và sứ mệnh như thế nào? Dưới đây là thông tin liên quan:

Tầm nhìn:
Tầm nhìn của MetaDAP là xây dựng nền tảng tài sản số dành cho doanh nghiệp (Enterprise Blockchain), giúp doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của blockchain nhưng vẫn đảm bảo tuân thủ pháp lý, từ đó người dùng có thể trực tiếp quản lý và giao dịch sơ cấp (Primary market) lẫn thứ cấp (Secondary market) các tài sản có giá trị (vô hình và hữu hình) tự động, trực tuyến mà không cần sự tham gia của các bên trung gian.

Sứ mệnh:
Sứ mệnh của MetaDAP là xây dựng nền tảng Enterprise Blockchain hoàn chỉnh, giúp doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của công nghệ blockchain nhưng vẫn đảm bảo tuân thủ pháp lý, từ đó người dùng có thể trực tiếp quản lý và giao dịch sơ cấp lẫn thứ cấp các tài sản có giá trị tự động, trực tuyến mà không cần sự tham gia của các bên trung gian. Điều này giúp tiết kiệm hơn 90% thời gian, 50% chi phí, tăng tính khả dụng, thanh khoản của t

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "tóm tắt các dịch vụ metaDAP cung cấp"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

MetaDAP cung cấp các dịch vụ sau:

### 1. Phát triển nền tảng (Custom Development)

Phát triển nền tảng tùy chỉnh để đáp ứng nhu cầu cụ thể của doanh nghiệp, từ visuals đến tính năng, mọi điều kiện có thể được tailor-made để phù hợp với mục tiêu hoạt động của bạn.

### 2. Nền tảng quản lý tài sản (Asset Management)

Dễ dàng quản lý, phân phối và theo dõi tài sản trên môi trường online/offline, từ đó tăng tính khả dụng và giá trị của tài sản.

### 3. Nền tảng thanh toán (Payment Gateway)

Thanh toán bằng tiền pháp định (Fiat) và tiền điện tử (Cryptocurrency), hỗ trợ các phương thức thanh toán phổ biến như Visa, Mastercard, PayPal, hay Bitcoin, Ethereum,...

### 4. Tương thích EVM (Ethereum Virtual Machine)

Đội ngũ MetaDAP có nhiều năm kinh nghiệm trong lĩnh vực phát triển blockchain, đặc biệt là nền tảng Ethereum. MetaDAP phát triển các dự án trên nền EVM, đảm bảo sự tương thích và dễ dàng kết nối với hệ sinh thái Ethereum.

### 5. Consulting và Đào tạo

Chuyên gia MetaDAP sẽ giúp bạn 

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Ban điều hành Viện gồm những ai"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 500, pad_token_id = tokenizer.eos_token_id)

Ban điều hành Viện gồm những ai?

Tại Việt Nam:

* Viện trưởng: TS. Trần Quý
* Phó Viện trưởng Phụ trách Đối Ngoại và Hợp tác Quốc Tế: Bà Bùi Thị Thuỷ Tiên
* Phó Viện trưởng Phụ trách Chiến lược và Chính sách: TS. Trần Đức Thanh
* Phó Viện trưởng Phụ trách Vừa và Nhỏ: TS. Nguyễn Thế Kiên
* Phó Viện trưởng Phụ trách Đào Tạo và liên kết: ThS. Nguyễn Vũ Thắng<|eot_id|>


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Viện cung cấp các khoá học gì"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 500, pad_token_id = tokenizer.eos_token_id)

Viện trưởng có thể cung cấp các khoá học sau:

1. Khoá đào tạo chuyên đề: Khoá đào tạo chuyên đề được tổ chức trong thời gian ngắn (vượt ra khỏi thời gian thường nhật) với mục đích truyền tải thông tin, kiến thức mới để đáp ứng nhu cầu công việc hiện tại hoặc tương lai.

2. Khoá đào tạo nâng cao năng lực: Khoá đào tạo nâng cao năng lực được tổ chức trong thời gian dài hơn, có thể từ vài ngày đến vài tháng, với mục đích giúp các nhân, tổ chức nâng cao năng lực hoạt động, thích ứng với môi trường kinh doanh đang mạnh mẽ thay đổi.

3. Khoá đào tạo chuyên ngành: Khoá đào tạo chuyên ngành được tổ chức trong thời gian dài, có thể từ vài tháng đến vài năm, với mục đích giúp các nhân hoàn thành việc đào tạo chuyên ngành, nâng cao năng lực chuyên môn và mở ra những cơ hội mới trên thị trường lao động.

4. Khoá đào tạo kết hợp lý thuyết và thực hành: Khoá đào tạo kết hợp lý thuyết và thực hành được tổ chức với mục đích giúp học viên không chỉ hiểu biết về vấn đề mới mà còn có thể thực hiện việc 

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Chi tiết khoá học bản chất blockchain và ứng dụng thực tiễn "},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

Khoá học "Blockchain Bản Chất và Ứng dụng Thực Tiên" cung cấp kiến thức sâu rộng về công nghệ Blockchain, từ bản chất công nghệ đến ứng dụng thực tiễn. Trong khoá học này, bạn sẽ học được:

### Bản chất Blockchain

1. Lịch sử phát triển của Blockchain
2. Công nghệ đảm bảo sự minh bạch, bất biến và khả năng truy vết lịch sử cập nhật của Blockchain là gì
3. Cơ chế hoạt động của Blockchain, gồm những bước nào

### Các loại Blockchain

1. Public Blockchain: Ethereum, Bitcoin...
2. Private Blockchain: Hyperledger Fabric, Corda...
3. Hybrid Blockchain: Blend của Public và Private Blockchain

### ứng dụng thực tiễn của Blockchain

1. Tài sản số: Các mô hình triển khai tài sản số sử dụng Blockchain để quản lý và giao dịch
2. Thanh toán số: Phương thức thanh toán sử dụng Blockchain để quản lý và giao dịch
3. Quản lý nguồn nhân lực: Sử dụng Blockchain để quản lý và giao dịch các dịch vụ liên quan đến nhân lực
4. Quản lý tài sản: Quản lý và giao dịch các tài sản có giá trị cao, như cổ phiếu, đất 

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "VIDE là gì "},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 500, pad_token_id = tokenizer.eos_token_id)

VIDE là viết tắt của Viện Phát triển Kinh tế số Việt Nam.<|eot_id|>


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Những bất cập trong vấn đề crowfunding tại việt nam"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 10800, pad_token_id = tokenizer.eos_token_id)

Crowfunding (Kêu gọi vốn cộng đồng) là mô hình tài chính mới, cho phép doanh nghiệp, tổ chức hoặc cá nhân kêu gọi vốn từ nhiều người khác thông qua nền tảng online. Tại Việt Nam, crowfunding vẫn đang trong quá trình phát triển, với những bất cập chính sách, pháp lý và thực tiễn vận hành, làm giảm sự hấp dẫn cho các bên muốn tham gia. Dưới đây là những bất cập chính:

1. **Chưa có luật riêng**: Việt Nam chưa có luật riêng về crowfunding, dẫn tới sự thiếu minh bạch và an toàn, làm giảm sự tự tin cho các nhà đầu tư và người kêu gọi vốn.
2. **Trải qua nhiều cơ quan quản lý**: Crowfunding tại Việt Nam được quản lý bởi các ban ngành khác nhau, bao gồm Bộ Tài chính, Ngân hàng Trung ương, và các văn phòng quản lý thị trường (OA), tạo ra sự phức tạp và chậm chạp trong quá trình triển khai.
3. **Giới hạn cho các tổ chức pháp lý**: Chỉ các tổ chức pháp lý được phép tổ chức crowfunding, điều này hạn chế sự tham gia của các cá nhân, nhà tư vấn và các tổ chức khác có thể góp phần tạo điều kiện thuận

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Tạo câu slogan cho metaDAP"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 500, pad_token_id = tokenizer.eos_token_id)

MetaDAP là một công ty khởi nghiệp công nghệ, phát triển nền tảng tài sản số dành cho doanh nghiệp (Enterprise Blockchain), cho phép doanh nghiệp dễ dàng xây dựng các ứng dụng khai thác lợi thế của blockchain nhưng vẫn đảm bảo tuân thủ pháp lý, từ đó người dùng có thể trực tiếp quản lý và giao dịch sơ cấp (Primary market) lẫn thứ cấp (Secondary market) các tài sản có giá trị (vô hình và hữu hình) tự động, trực tuyến mà không cần sự tham gia của các bên trung gian. Dưới đây là một vài ý tưởng slogan cho MetaDAP:

1) "Tài sản số cho doanh nghiệp: Tiếp cận dễ dàng, quản lý trực tuyến"
2) "Đổi mới nền kinh tế: Tài sản số cho mọi nhu cầu"
3) "Quy trình giao dịch tiết kiệm thời gian: Chỉ với vài cú nhấp chuột"
4) "Tài sản số chính thức bước vào kỷ nguyên số: MetaDAP dẫn đầu"
5) "Giao dịch sơ cấp và thứ cấp liền mạch, trực tuyến mà không có trung gian"
6) "Đội ngũ tư vấn chuyên sâu: Phụ hợp với luật pháp và thực tiễn kinh doanh"
7) "MetaDAP Enterprise Blockchain: Viễn tưởng trở thành thực tế"

<a name="Save"></a>
# Lưu và tải model sau train
Bên dưới cung cấp các hình thức lưu model sau train gồm lưu một phần nhỏ với LoRA và lưu cả mô hình 16bit

## Lưu dưới dạng LoRA tiết kiệm dung lượng

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

## Lưu full model dưới dạng GGUF để sử dụng trực tiếp với Ollama
Gồm các cấu hình khác nhau
f16 (16-bit Floating Point - Full Precision)

- Dữ liệu lưu trữ ở dạng 16-bit float.

- Độ chính xác cao nhất nhưng dung lượng lớn.

- Cần GPU mạnh hoặc CPU hỗ trợ AVX/FMA.

q8_0 (8-bit Quantization)

- Dữ liệu nén xuống 8-bit (mỗi giá trị được lưu trong 8-bit thay vì 16-bit hoặc 32-bit).

- Cân bằng giữa tốc độ và độ chính xác.

- Dùng được trên CPU thông thường.

q4_k_m (4-bit Quantization - K-Method)

- Dữ liệu nén xuống 4-bit.

- Tốc độ nhanh nhất, nhưng có thể mất một chút chính xác.

- Dành cho thiết bị có RAM thấp hoặc CPU yếu.

q5_k_m (5-bit Quantization - K-Method)

- Giữa 4-bit và 8-bit, có độ chính xác tốt hơn q4_k_m nhưng nhẹ hơn q8_0.

- Tốt nếu bạn cần tốc độ nhanh nhưng vẫn có độ chính xác tốt.

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.01 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 13.31it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [03:25<00:00,  6.42s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {4

## Đẩy model lên repository hugging face

In [ ]:
import os
os.environ["HF_TOKEN"] = "" # Nhập token huggingface

from huggingface_hub import HfApi

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="/content/model", # Thay thành repo đã tạo trên huggingface
    repo_id="hoanghuy100202/LLAMA3_FineTuning_Vide_MetaDAP",
    repo_type="model",
)

  0%|          | 0/6 [00:00<?, ?it/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hoanghuy100202/LLAMA3_FineTuning_Vide_MetaDAP/commit/16c38b5d042331a55d41bf663d22d9d231618d26', commit_message='Upload folder using huggingface_hub', commit_description='', oid='16c38b5d042331a55d41bf663d22d9d231618d26', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hoanghuy100202/LLAMA3_FineTuning_Vide_MetaDAP', endpoint='https://huggingface.co', repo_type='model', repo_id='hoanghuy100202/LLAMA3_FineTuning_Vide_MetaDAP'), pr_revision=None, pr_num=None)